In [1]:
# 텐서플로우 모델 생성 위한 import

import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
tf.set_random_seed(777)

# 초기값 선정 xavier 알고리즘
def xavier_init(n_inputs, n_outputs, uniform=True):
    
    if uniform:
        init_range = tf.sqrt(6.0 / (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)

# 색온도 학습 및 테스트 (편차 분포)

import_data = np.genfromtxt('./../swr_area_test.csv', delimiter=',', dtype='float')

x_data = import_data[:, :2]
scaler = MinMaxScaler(feature_range=(0, 1))
x_data = scaler.fit_transform(x_data)
test_x = x_data[86:, :2]
x_data = x_data[:86, :2]

y_data = []
test_y = []

for i in range(86):
    temp = []
    temp.append(import_data[i][2])
    y_data.append(temp)

raw_y = y_data
    
for i in range(86, len(import_data)):
    temp = []
    temp.append(import_data[i][2])
    test_y.append(temp)
    
# raw_y = y_data

y_data = ohe.fit_transform(y_data)
y_data = y_data.toarray();

X = tf.placeholder("float", [None, 2])
Y = tf.placeholder("float", [None, 5])

nb_classes = 5

#기존 코드
# W = tf.Variable(tf.random_normal([11, nb_classes]), name='weight')
# b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# Xavier Initializer 추가 코드
W = tf.get_variable("W", shape=[2, nb_classes], initializer=xavier_init(2, nb_classes))
b = tf.Variable(tf.zeros([nb_classes]))

H = tf.nn.softmax(tf.matmul(X, W) + b)

#  dropout 추가 코드
# dropout_rate = tf.placeholder("float")

# W1 = tf.get_variable("W1", shape=[11, 30], initializer=xavier_init(11, 30))
# W2 = tf.get_variable("W2", shape=[30, 30], initializer=xavier_init(30, 30))
# W3 = tf.get_variable("W3", shape=[30, nb_classes], initializer=xavier_init(30, nb_classes))

# B1 = tf.Variable(tf.random_normal([30]))
# B2 = tf.Variable(tf.random_normal([30]))
# B3 = tf.Variable(tf.random_normal([5]))

# _L1 = tf.nn.relu(tf.add(tf.matmul(X, W1), B1))
# L1 = tf.nn.dropout(_L1, dropout_rate)
# _L2 = tf.nn.relu(tf.add(tf.matmul(_L1, W2), B2))
# L2 = tf.nn.dropout(_L2, dropout_rate)

# H = tf.add(tf.matmul(_L2, W3), B3)

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(H), axis=1))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

with tf.Session() as sess:
    accuracy = 0
    sess.run(tf.global_variables_initializer())

    for step in range(6401):
        sess.run(optimizer, feed_dict={X:x_data, Y:y_data})

        if step % 100 == 0:
            print(step, sess.run(cost, feed_dict={X:x_data, Y:y_data}))

    print('-----------------------------')

    print('train_data = ', len(x_data), 'test_data = ', len(test_x))

    for i in range(len(test_x)):
        a = sess.run(H, feed_dict={X:[test_x[i]]})
        print(sess.run(tf.argmax(a, 1)), test_y[i])
        if(sess.run(tf.argmax(a, 1)) ==test_y[i]):
            accuracy += 1
    print("accuracy = ", float(accuracy / len(test_x) * 100))

0 1.2960602
100 1.2277786
200 1.1997769
300 1.1747309
400 1.1514864
500 1.1298746
600 1.109834
700 1.0912719
800 1.074065
900 1.0580753
1000 1.0431656
1100 1.0292066
1200 1.0160819
1300 1.0036875
1400 0.9919336
1500 0.98074216
1600 0.97004646
1700 0.95978934
1800 0.9499223
1900 0.94040394
2000 0.9311994
2100 0.9222786
2200 0.9136168
2300 0.90519226
2400 0.8969873
2500 0.88898647
2600 0.8811769
2700 0.87354755
2800 0.86608905
2900 0.85879296
3000 0.85165244
3100 0.8446611
3200 0.83781326
3300 0.8311042
3400 0.82452923
3500 0.8180843
3600 0.8117654
3700 0.80556875
3800 0.799491
3900 0.7935291
4000 0.78767943
4100 0.7819394
4200 0.7763056
4300 0.77077574
4400 0.7653469
4500 0.7600163
4600 0.7547815
4700 0.74964
4800 0.74458945
4900 0.7396274
5000 0.7347517
5100 0.7299601
5200 0.7252503
5300 0.7206206
5400 0.71606857
5500 0.71159256
5600 0.7071906
5700 0.7028607
5800 0.6986012
5900 0.6944104
6000 0.6902865
6100 0.68622786
6200 0.68223286
6300 0.67829996
6400 0.6744278
---------------------